In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     -------------------------------------- 101.4/101.4 kB 5.7 MB/s eta 0:00:00


In [40]:
import pandas as pd
import numpy as np
import asyncio, requests, websockets
import os, sys, random, json

from wazirx_sapi_client.rest import Client
from wazirx_sapi_client.websocket import WebsocketClient

from tqdm import tqdm
from configparser import ConfigParser
from datetime import datetime

from kafka import KafkaProducer
from time import sleep
from json import dumps

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read Config File
cfg_file = 'config.ini'
config = ConfigParser(interpolation=None)
config.read(cfg_file, encoding='utf-8')

# Storing the contents of the config file into respective dictionary variables
wazirx_keys = dict(config.items('wazirx-keys'))

In [19]:
client = Client(api_key=wazirx_keys['api_key'], secret_key=wazirx_keys['secret_key'])
print(client.send("ping"))

(200, {})


In [20]:
tickers = client.send('tickers')[1]

In [6]:
pd.to_datetime(at, unit='ms')

Timestamp('2023-02-05 00:06:49')

In [23]:
df = pd.DataFrame(tickers)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439 entries, 0 to 438
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   symbol      439 non-null    object
 1   baseAsset   439 non-null    object
 2   quoteAsset  439 non-null    object
 3   openPrice   439 non-null    object
 4   lowPrice    439 non-null    object
 5   highPrice   439 non-null    object
 6   lastPrice   439 non-null    object
 7   volume      439 non-null    object
 8   bidPrice    439 non-null    object
 9   askPrice    439 non-null    object
 10  at          439 non-null    int64 
dtypes: int64(1), object(10)
memory usage: 37.9+ KB


In [26]:
df['openPrice'] = df['openPrice'].astype(float)
df['lowPrice']  = df['lowPrice'].astype(float)
df['highPrice'] = df['highPrice'].astype(float)
df['lastPrice'] = df['lastPrice'].astype(float)
df['volume']    = df['volume'].astype(float)
df['bidPrice']  = df['bidPrice'].astype(float)
df['askPrice']  = df['askPrice'].astype(float)

In [28]:
df['datetime'] = pd.to_datetime(df['at'], unit='ms')

In [30]:
df

,symbol,baseAsset,quoteAsset,openPrice,lowPrice,highPrice,lastPrice,volume,bidPrice,askPrice,at,datetime
0,btcinr,btc,inr,1.953004e+06,1.935000e+06,1.987512e+06,1.951010e+06,1.313436e+01,1.951010e+06,1.967814e+06,1675556482000,2023-02-05 00:21:22
1,xrpinr,xrp,inr,3.400010e+01,3.350340e+01,3.490000e+01,3.400150e+01,2.515998e+05,3.400120e+01,3.478990e+01,1675556482000,2023-02-05 00:21:22
2,ethinr,eth,inr,1.394950e+05,1.353335e+05,1.405320e+05,1.399880e+05,7.493090e+01,1.382070e+05,1.399880e+05,1675556482000,2023-02-05 00:21:22
3,trxinr,trx,inr,5.327400e+00,5.265500e+00,5.450000e+00,5.395000e+00,1.923986e+06,5.300800e+00,5.400300e+00,1675556482000,2023-02-05 00:21:22
4,eosinr,eos,inr,9.279000e+01,9.003000e+01,9.279000e+01,9.100000e+01,7.549300e+02,9.100000e+01,9.240000e+01,1675556482000,2023-02-05 00:21:22
...,...,...,...,...,...,...,...,...,...,...,...,...
434,1inchwrx,1inch,wrx,2.700000e+00,2.700000e+00,2.986000e+00,2.939000e+00,1.778600e+02,2.839000e+00,2.990000e+00,1675556484000,2023-02-05 00:21:24
435,dockwrx,dock,wrx,1.000000e-01,1.000000e-01,1.000000e-01,1.000000e-01,7.610000e+03,9.000000e-02,1.000000e-01,1675556484000,2023-02-05 00:21:24
436,runewrx,rune,wrx,9.598000e+00,9.111000e+00,9.598000e+00,9.111000e+00,7.850000e-01,8.775000e+00,9.594000e+00,1675556484000,2023-02-05 00:21:24
437,chrwrx,chr,wrx,9.292000e-01,8.523000e-01,9.292000e-01,9.270000e-01,9.400000e+01,8.002000e-01,9.269000e-01,1675556484000,2023-02-05 00:21:24


In [32]:
# To make sure the websocket stays live
loop = asyncio.get_event_loop()

In [33]:
loop.

<_WindowsSelectorEventLoop running=True closed=False debug=False>

In [34]:
len(df)

439

In [35]:
pd.DataFrame(client.send('tickers')[1])

,symbol,baseAsset,quoteAsset,openPrice,lowPrice,highPrice,lastPrice,volume,bidPrice,askPrice,at
0,btcinr,btc,inr,1952608,1935000.0,1984998.0,1961999.0,13.13436,1942000.0,1961998.0,1675560013000
1,xrpinr,xrp,inr,34.3998,33.5034,34.9,34.0012,251599.8,34.0012,34.7778,1675560013000
2,ethinr,eth,inr,137200,135333.5,140532.0,139499.0,74.9309,138000.1,139499.0,1675560013000
3,trxinr,trx,inr,5.29,5.2655,5.45,5.3432,1923986.0,5.3432,5.3999,1675560013000
4,eosinr,eos,inr,92.79,90.03,92.79,91.0,754.93,91.0,92.4,1675560013000
...,...,...,...,...,...,...,...,...,...,...,...
434,1inchwrx,1inch,wrx,2.7,2.7,3.0,3.0,177.86,2.839,3.0,1675560016000
435,dockwrx,dock,wrx,0.1,0.1,0.1,0.1,7610.0,0.09,0.1,1675560016000
436,runewrx,rune,wrx,9.598,9.111,9.598,9.111,0.785,8.775,9.594,1675560016000
437,chrwrx,chr,wrx,0.9292,0.8523,0.9292,0.927,94.0,0.8002,0.9269,1675560016000


In [39]:
df.sample(1).to_dict(orient="records")[0]

{'symbol': 'zilusdt',
 'baseAsset': 'zil',
 'quoteAsset': 'usdt',
 'openPrice': 0.02915,
 'lowPrice': 0.02794,
 'highPrice': 0.03199,
 'lastPrice': 0.03199,
 'volume': 47491.0,
 'bidPrice': 0.02901,
 'askPrice': 0.03199,
 'at': 1675556483000,
 'datetime': Timestamp('2023-02-05 00:21:23')}

In [43]:
producer = KafkaProducer(bootstrap_servers=['18.119.158.111:9092'],
                             value_serializer=lambda x: dumps(x).encode('utf-8'))

In [45]:
while True:
    dict_tickers = df.sample(1).to_dict(orient="records")[0]
    producer.send('crypto_stream', value=dict_tickers)
    print()

TypeError: Object of type Timestamp is not JSON serializable